In [ ]:
import json
from datetime import datetime,timedelta
from dateutil.relativedelta import relativedelta
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import interp1d

def loadingDataAux(fileAddress,parameter):
    try:
        jsonFile = open(fileAddress, )
        data = json.load(jsonFile)

        dateTemp=[]
        valTemp=[]
        for i in data:
            for j in data[i]:
                varTemp=j["time"]
                timeTemp=int(varTemp[0:2])/24+int(varTemp[3:5])/24/60+int(varTemp[7:9])/24/60/60

                varTemp=i
                varTemp = datetime.strptime(varTemp, '%m-%d')+relativedelta(years=+121)
                varTemp=varTemp.timetuple().tm_yday
                dateTemp.append(varTemp+timeTemp)
                valTemp.append(float(j["value"]))
        jsonFile.close()
    except:
        print("An exception occurred")

    parTemp = []
    for i in range (0,len(dateTemp)):
        parTemp.append(parameter)

    return dateTemp,valTemp,parTemp

def loadingData():
    if not os.path.exists(r'C:\GitHub\BobakData\Result.csv'):
        allDate=[]
        allVal=[]
        allPar=[]

        fileAddress=r'C:\GitHub\BobakData\Processed Data\intraday-ACC_E4.json'
        [tempDate,tempVal,tempPar]=loadingDataAux(fileAddress,'Acc')
        allDate.extend(tempDate)
        allVal.extend(tempVal)
        allPar.extend(tempPar)

        fileAddress=r'C:\GitHub\BobakData\Processed Data\intraday-calories.json'
        [tempDate,tempVal,tempPar]=loadingDataAux(fileAddress,'Cal')
        allDate.extend(tempDate)
        allVal.extend(tempVal)
        allPar.extend(tempPar)

        fileAddress=r'C:\GitHub\BobakData\Processed Data\intraday-EDA_E4.json'
        [tempDate,tempVal,tempPar]=loadingDataAux(fileAddress,'EDA')
        allDate.extend(tempDate)
        allVal.extend(tempVal)
        allPar.extend(tempPar)

        fileAddress=r'C:\GitHub\BobakData\Processed Data\intraday-HR_E4.json'
        [tempDate,tempVal,tempPar]=loadingDataAux(fileAddress,'HR')
        allDate.extend(tempDate)
        allVal.extend(tempVal)
        allPar.extend(tempPar)

        fileAddress=r'C:\GitHub\BobakData\Processed Data\intraday-steps.json'
        [tempDate,tempVal,tempPar]=loadingDataAux(fileAddress,'Step')
        allDate.extend(tempDate)
        allVal.extend(tempVal)
        allPar.extend(tempPar)

        fileAddress=r'C:\GitHub\BobakData\Processed Data\intraday-TEMP_E4.json'
        [tempDate,tempVal,tempPar]=loadingDataAux(fileAddress,'Temp')
        allDate.extend(tempDate)
        allVal.extend(tempVal)
        allPar.extend(tempPar)

        fileAddress=r'C:\GitHub\BobakData\Processed Data\intraday-glucose.json'
        [tempDate,tempVal,tempPar]=loadingDataAux(fileAddress,'CGM')
        allDate.extend(tempDate)
        allVal.extend(tempVal)
        allPar.extend(tempPar)

        df = pd.DataFrame(list(zip(allDate, allPar,allVal)),columns =['Date','Parameter','Value'])
        df.to_csv(r'C:\GitHub\BobakData\Result.csv', header=True)
    else:
        df=pd.read_csv (r'C:\GitHub\BobakData\Result.csv')
    return df

df=loadingData()

i = df[((df.Parameter == 'Temp') &( df.Value < 30))].index
df=df.drop(i)

days=df['Date']
days=days.values.tolist()
days=np.asarray(days,dtype=int)

days=set(days)
days=list(days)
days.sort()

del days[0]
del days[0]
del days[-1]
del days[-1]

for day in days:
    fig, ax1 = plt.subplots ()
    ax2 = ax1.twinx ()

    inquery=df[df['Date']>=day]
    inquery=inquery[inquery['Date']<day+1]
    inquery=inquery[inquery['Parameter']=='CGM']

    x=inquery['Date']
    x=x.values.tolist()
    x=np.asarray(x,dtype=float)

    y=inquery['Value']
    y=y.values.tolist()
    y=np.asarray(y,dtype=float)

    ax1.plot(x,y,color='b')
    ax1.tick_params (axis='y', labelcolor='b')

    # inquery=df[df['Date']>=day]
    # inquery=inquery[inquery['Date']<day+1]
    # inquery=inquery[inquery['Parameter']=='Temp']
    # ax2.plot (inquery ['Date'], inquery ['Value'],color='r')
    # ax2.tick_params (axis='y', labelcolor='r')
    f = interp1d (x, y)
    xnew=np.arange(min(x),max(x),0.001)
    ax1.plot(xnew,f(xnew),color='red')
    plt.show()

    break



